In [1]:
import os

import numpy as np
import torch
from lib.config import cfg

import lib.dataset.dataset3D as dataUtils
from lib.hybridnet.hybridnet import HybridNet
from lib.hybridnet.efficienttrack.efficienttrack import EfficientTrack
import lib.hybridnet.efficienttrack.darkpose as darkpose
from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Test_Project')

#project.create_new(
#    name = 'CSV Creation',
#    dataset2D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen',
#    dataset3D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen')

cfg = project.get_cfg()

Successfully loaded project Test_Project!


In [2]:
training_set = dataUtils.Dataset3D(cfg, set='train')
val_set = dataUtils.Dataset3D(cfg, set='val')

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
hybridNet = HybridNet('inference', cfg, '/home/timo/Desktop/HybridNet/projects/Test_Project/models/HybridNet/Run_20211120-195826/HybridNet-d_20.pth')

[Info] loaded weights: HybridNet-d_20.pth


In [4]:
weights = "/home/timo/Desktop/HybridNet/projects/Ralph_Center_Test/models/efficienttrack/EfficientTrack-d0_210.pth"

centerDetect = EfficientTrack('CenterDetectInference', cfg, weights)

In [8]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

    

video_paths = [
    '/media/timo/Elements/Example_Recording/Camera_B.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LBB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LBT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LC.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LFB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LFT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RBB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RBT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RC.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RFB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RFT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_T.mp4', 
]


caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    outs.append(cv2.VideoWriter(os.path.join('/home/timo/Desktop/HybridNet/Videos_Ralph_Center', path.split('/')[-1].split(".")[0] + ".avi"), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0) #99640
        
def process(cap):
    ret, img = cap.read()
    return img
        
counter = 0
    
with open('3D Data/matthias.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ['Frame Idx']
    joints = ["Pinky_T","Pinky_D","Pinky_M","Pinky_P","Ring_T","Ring_D","Ring_M","Ring_P","Middle_T","Middle_D","Middle_M","Middle_P","Index_T","Index_D","Index_M","Index_P","Thumb_T","Thumb_D","Thumb_M","Thumb_P","Palm", "Wrist_U","Wrist_R"]
    joints = joints
    joints = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in joints))
    header = header + joints
    header2 = [""]
    header2 = header2 + ['x','y', 'z']*23
    writer.writerow(header)
    writer.writerow(header2)

    ret = True
    while ret:
        if counter % 20 == 0:
            print (counter)
        counter += 1
        imgs = []
        imgs_orig = []
        centerHMs = []
        camsToUse = []
        
        imgs_orig = Parallel(n_jobs=12, require='sharedmem')(delayed(process)(cap) for cap in caps)
        imgs = torch.zeros(12,3,256,320)
        for i,img in enumerate(imgs_orig[:]):
            img = ((cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0-cfg.DATASET.MEAN)/cfg.DATASET.STD)
            img = cv2.resize(img, (320,256))
            imgs[i] = torch.from_numpy(cv2.resize(img, (320,256)).transpose(2,0,1))

        imgs = imgs.cuda()
        outputs = centerDetect.model(imgs)
        preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)
        camsToUse = []

        for i,val in enumerate(maxvals[:]):
            if val > 180:
                camsToUse.append(i)

        if len(camsToUse) >= 2:
            center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
            reproPoints = val_set.reproTool.reprojectPoint(center3D)

            errors = []
            errors_valid = []
            for i in range(12):
                if maxvals[i] > 180:
                    errors.append(np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i]))
                    errors_valid.append(np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i]))
                else:
                    errors.append(0)
            medianError = np.median(np.array(errors_valid))
            print ("Error: ", medianError)
            print ("Var based: ", medianError+2*np.sqrt(np.var(errors_valid)),medianError*4)
            camsToUse = []
            for i,val in enumerate(maxvals[:]):
                if val > 180 and errors[i] < 2*medianError:
                    camsToUse.append(i)
            center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
            reproPoints = val_set.reproTool.reprojectPoint(center3D)
        imgs = []
        bbox_hw = int(256/2)
        for idx,reproPoint in enumerate(reproPoints):
            reproPoint = reproPoint.astype(int)
            img = imgs_orig[idx][reproPoint[1]-bbox_hw:reproPoint[1]+bbox_hw, reproPoint[0]-bbox_hw:reproPoint[0]+bbox_hw, :]
            img = ((cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0-cfg.DATASET.MEAN)/cfg.DATASET.STD)
            imgs.append(img)

        if not ret:
            break
        imgs = torch.from_numpy(np.array(imgs))
        imgs = imgs.permute(0,3,1,2).view(1,12,3,256,256).cuda().float()
        centerHMs = np.array(reproPoints).astype(int)
        if len(camsToUse) >= 2:
            center3D = center3D.int().cuda()
            centerHMs = torch.from_numpy(centerHMs).cuda()
            heatmap3D, heatmaps_padded, points3D_net = hybridNet.model(imgs, torch.unsqueeze(centerHMs,0), torch.unsqueeze(center3D, 0), torch.unsqueeze(val_set.cameraMatrices.cuda(),0))
            row = [counter]
            for point in points3D_net.squeeze():
                row = row + point.tolist()
            writer.writerow(row)
            points2D = []
            for point in points3D_net[0].cpu().numpy():
                points2D.append(val_set.reproTool.reprojectPoint(point))

            colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                      (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

        else:
            row = [counter]
            for i in range(23*3):
                row = row + ['NaN']
            writer.writerow(row)
            
        for i,out in enumerate(outs):
            img = imgs_orig[i]
            if len(camsToUse) >= 2:
                for j,point in enumerate(points2D):
                    cv2.circle(img, (int(point[i][0]), int(point[i][1])), 2, colors[j], thickness=5)
            out.write(img)
            

    

0
Error:  4.043244804643754
Var based:  8.458593977335024 16.172979218575016
Error:  4.552894069207638
Var based:  8.652528757821262 18.211576276830552
Error:  4.411844995543832
Var based:  8.504866652076938 17.647379982175327
Error:  4.479617458126379
Var based:  8.518594662475081 17.918469832505515
Error:  4.321210284888928
Var based:  8.43048675642892 17.284841139555713
Error:  4.366236195224043
Var based:  8.282400584822522 17.46494478089617
Error:  4.45872202498241
Var based:  8.369149967901443 17.83488809992964
Error:  4.466352318122926
Var based:  8.378259343104727 17.865409272491704
Error:  4.473053984355751
Var based:  8.360437966750869 17.892215937423003
Error:  4.802625462065858
Var based:  8.733584055778588 19.210501848263434
Error:  4.721016364088294
Var based:  8.625971483927273 18.884065456353177
Error:  4.482176757801329
Var based:  8.36226479489946 17.928707031205317
Error:  4.4583083157603784
Var based:  8.337519264648854 17.833233263041514
Error:  4.547312641552649
V

Error:  5.507963732114009
Var based:  9.06398065800367 22.031854928456035
Error:  5.509177592788877
Var based:  9.04004222888563 22.036710371155507
Error:  5.545772800537592
Var based:  9.123281740151974 22.18309120215037
Error:  5.569670847213149
Var based:  9.009643479553617 22.278683388852595
Error:  5.582540862832487
Var based:  9.02771652461816 22.33016345132995
Error:  5.556090332128302
Var based:  8.944102915868696 22.224361328513208
Error:  5.427480898596224
Var based:  8.833722926448091 21.709923594384897
Error:  5.44060815985438
Var based:  8.725689921403527 21.76243263941752
Error:  5.46476897320211
Var based:  8.652430669039752 21.85907589280844
Error:  5.619848576640258
Var based:  10.024189128749194 22.479394306561034
120
Error:  5.699447850053555
Var based:  10.146377525522226 22.79779140021422
Error:  5.6860734625080855
Var based:  10.161713035658638 22.744293850032342
Error:  5.7441877764743765
Var based:  10.19611417173246 22.976751105897506
Error:  5.609748906969731


220
Error:  3.2019502977374694
Var based:  6.99698174638023 12.807801190949878
Error:  3.2218367528920737
Var based:  6.988289942923306 12.887347011568295
Error:  3.364944061985177
Var based:  7.122131129972055 13.459776247940708
Error:  3.1732733542302483
Var based:  7.047169127766019 12.693093416920993
Error:  3.1004843420787243
Var based:  7.023296992392536 12.401937368314897
Error:  3.163206744823595
Var based:  7.004814553552425 12.65282697929438
Error:  3.105382621825241
Var based:  6.9905460162422335 12.421530487300965
Error:  3.1663539233832374
Var based:  6.981213439186949 12.66541569353295
Error:  3.0673262256964433
Var based:  6.970020790785038 12.269304902785773
Error:  3.4890845988045327
Var based:  7.2285490104773125 13.95633839521813
Error:  3.7381582090754093
Var based:  7.403448889352212 14.952632836301637
Error:  3.9086947949496906
Var based:  7.499365387189645 15.634779179798763
Error:  3.908247146026449
Var based:  7.473574902789183 15.632988584105796
Error:  3.5993

Error:  4.2491240045720975
Var based:  9.370545560940549 16.99649601828839
Error:  4.191030874599571
Var based:  9.398051720640506 16.764123498398284
Error:  4.189327656251329
Var based:  9.394940669561738 16.757310625005317
Error:  4.171459809157563
Var based:  9.379074074429582 16.68583923663025
Error:  4.341869354238124
Var based:  9.448702149372465 17.367477416952497
Error:  4.329621083556087
Var based:  9.592505522288516 17.318484334224348
Error:  4.405191886838921
Var based:  9.621542100900594 17.620767547355683
Error:  4.3750843152918115
Var based:  9.621543259351313 17.500337261167246
Error:  4.475220385676004
Var based:  9.674394583429452 17.900881542704017
Error:  4.485524616476614
Var based:  9.700337620336764 17.942098465906454
340
Error:  4.521214251481826
Var based:  9.724308547984968 18.084857005927304
Error:  4.495790140928839
Var based:  9.72280037147624 17.983160563715355
Error:  4.512702330978854
Var based:  9.721539697865705 18.050809323915416
Error:  4.539335611425

440
Error:  3.78980454748532
Var based:  9.87146915804103 15.15921818994128
Error:  3.778887239843723
Var based:  9.843351116881605 15.115548959374893
Error:  3.559819455040815
Var based:  9.661054472595113 14.23927782016326
Error:  3.5105251574198557
Var based:  9.6138440019512 14.042100629679423
Error:  3.58175511670183
Var based:  9.586551274774635 14.32702046680732
Error:  3.5001288444528513
Var based:  9.522138862643724 14.000515377811405
Error:  3.4347559208147835
Var based:  9.411238710815248 13.739023683259134
Error:  3.3357796376442437
Var based:  9.321131090093449 13.343118550576975
Error:  3.340829539671618
Var based:  9.2719742076623 13.363318158686472
Error:  3.2852365288567182
Var based:  9.192510263633581 13.140946115426873
Error:  3.2936941424371224
Var based:  9.169616319240191 13.17477656974849
Error:  3.3974271071547797
Var based:  9.274995948666996 13.589708428619119
Error:  3.6186349163884755
Var based:  9.530692820887452 14.474539665553902
Error:  3.69993684234466

...

Error:  6.71835050511525
Var based:  10.784171178216159 26.873402020461
Error:  6.76431946116931
Var based:  10.878431377131495 27.05727784467724
Error:  6.727637592407428
Var based:  10.72923987393095 26.91055036962971
Error:  6.784890713238116
Var based:  10.570223076419708 27.139562852952466
Error:  6.789206423808304
Var based:  10.608081341765278 27.156825695233216
Error:  6.766504311393242
Var based:  10.69289696549263 27.06601724557297
Error:  6.744111112766743
Var based:  10.694226229838204 26.97644445106697
Error:  6.366677489443951
Var based:  10.056541472614459 25.466709957775805
Error:  6.331379335290157
Var based:  9.992205264436922 25.325517341160626
Error:  6.019970003582334
Var based:  9.608730269082688 24.079880014329337
Error:  5.986430720943678
Var based:  9.613176110397367 23.94572288377471
Error:  6.013973247945052
Var based:  9.179723787321498 24.05589299178021
Error:  6.142770648274511
Var based:  9.32380792306673 24.571082593098044
Error:  6.26272771631256
Var ba

Error:  5.172967284478789
Var based:  7.3311883354702765 20.691869137915155
Error:  5.14357590608357
Var based:  7.28048942973275 20.57430362433428
Error:  5.172997000967093
Var based:  7.311664048270533 20.69198800386837
Error:  4.998809787530597
Var based:  7.087013718025986 19.995239150122387
1320
Error:  4.96819435635953
Var based:  7.074050133291833 19.87277742543812
Error:  5.004623220215264
Var based:  7.082374941588592 20.018492880861057
Error:  5.053632630397636
Var based:  7.225410571843126 20.214530521590543
Error:  4.85414990834269
Var based:  6.852137073092656 19.41659963337076
Error:  4.938156863365589
Var based:  6.904001554795531 19.752627453462356
Error:  4.85863697069483
Var based:  6.855903022104288 19.43454788277932
Error:  4.859552700236106
Var based:  6.960630556530154 19.438210800944425
Error:  4.681899979941457
Var based:  6.742083432767974 18.72759991976583
Error:  4.653716586708104
Var based:  6.78797504578217 18.614866346832414
Error:  4.7910943694174035
Var 

Error:  1.913811289283662
Var based:  6.30091970317055 7.655245157134648
Error:  2.020501635237151
Var based:  6.337415835242921 8.082006540948605
Error:  2.0384593564801454
Var based:  6.382225797093004 8.153837425920582
Error:  1.9897450947169015
Var based:  6.475083188492718 7.958980378867606
Error:  1.6465539111595813
Var based:  6.2067054775863095 6.586215644638325
Error:  1.5350898141023979
Var based:  6.001142541014399 6.1403592564095915
Error:  1.495704511672366
Var based:  5.967090766237429 5.982818046689464
Error:  1.4927461181129074
Var based:  5.990226354094286 5.970984472451629
Error:  1.5915657454776067
Var based:  6.042557522319751 6.366262981910427
Error:  1.6635887907982778
Var based:  6.106670231609067 6.654355163193111
Error:  1.6993892107411601
Var based:  6.092235869665348 6.7975568429646405
Error:  1.7362516196921498
Var based:  6.072541821748331 6.945006478768599
Error:  1.8187599395245917
Var based:  6.217100252863491 7.275039758098367
Error:  1.7412429178846378

Error:  6.110007586953975
Var based:  10.557666663484966 24.4400303478159
Error:  6.142151241762733
Var based:  10.880618072246499 24.56860496705093
Error:  6.210449690181047
Var based:  10.582381106555365 24.84179876072419
Error:  5.802285427498066
Var based:  10.16852640829822 23.209141709992263
Error:  6.192763724271506
Var based:  10.447728170063325 24.771054897086024
1540
Error:  6.324798773238895
Var based:  10.884192358761442 25.29919509295558
Error:  6.416886563755158
Var based:  11.087655119769966 25.667546255020632
Error:  5.921608113148109
Var based:  10.342111558103742 23.686432452592435
Error:  5.36563376199917
Var based:  9.858587206149528 21.46253504799668
Error:  5.231157592150496
Var based:  9.821622664819703 20.924630368601985
Error:  5.5721634617152915
Var based:  9.661981292786455 22.288653846861166
Error:  5.49394550086169
Var based:  9.266327460726062 21.97578200344676
Error:  6.038570874109149
Var based:  10.483570114750023 24.154283496436594
Error:  6.0397102900

KeyboardInterrupt: 

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

figure = plt.figure()
axes = figure.gca(projection='3d')

c = ['r', 'r','r','r','b','b','b','b','g','g','g','g', 'orange', 'orange','orange','orange', 'y','y','y','y','purple', 'purple','purple']
line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]
points3D = points3D_net[0].cpu().numpy()
center3D = center3D.cpu()
for i, point in enumerate(points3D):
    #print ("Classic:", i, point)
    if i != 20:
        axes.scatter(point[0], point[1], point[2], color = c[i])
for line in line_idxs:
    axes.plot([points3D[line[0]][0], points3D[line[1]][0]], [points3D[line[0]][1], points3D[line[1]][1]], [points3D[line[0]][2], points3D[line[1]][2]], c = 'gray')

axes.set_xlim3d(center3D[0]-100, center3D[0]+100)
axes.set_ylim3d(center3D[1]-100, center3D[1]+100)
axes.set_zlim3d(center3D[2]-100, center3D[2]+100)
plt.subplots_adjust(left=0., right=1., top=1., bottom=0.)
#plt.gca().invert_zaxis()
plt.savefig('test.png')
plt.show()